In [1]:
from sqlalchemy import create_engine

import pandas as pd

In [11]:
db_path = "barManolo.db"

engine = create_engine(f"sqlite:///{db_path}")

df_cervezas = pd.DataFrame({
    'CodC': ['01', '02', '03', '04', '05'],
    'Envase': ['Botella', 'Botella', 'Lata', 'Botella', 'Barril'],
    'Capacidad': [0.2, 0.33, 0.33, 1, 60],
    'Stock': [3600, 1200, 2400, 288, 30]
})
df_bares = pd.DataFrame({
    'CodB': ['001', '002', '003', '004'],
    'Nombre': ['Stop', 'Las Vegas', 'Club Social', 'Otra Ronda'],
    'Cif': ['11111111X', '22222222Y', '33333333Z', '33333333Z'],
    'Localidad': ['Villa Botijo', 'Villa Botijo', 'Las Ranas', 'La Esponja']
})

df_empleados = pd.DataFrame({
    'CodE': [1, 2, 3],
    'Nombre': ['Prudencio Caminero', 'Vicente Merario', 'Valentín Siempre'],
    'Sueldo': [120000, 110000, 100000]
})

df_reparto = pd.DataFrame({
    'CodE': [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3],
    'CodB': ['001', '001', '002', '002', '002', '002', '002', '002', '003', '003', '003'],
    'CodC': ['01', '02', '03', '01', '02', '03', '04', '05', '03', '04', '03'],
    'Fecha': ['10/21/05', '10/21/05', '10/22/05', '10/22/05', '10/22/05', '10/22/05',
              '10/23/05', '10/24/05', '10/24/05', '10/24/05', '10/25/05'],
    'Cantidad': [240, 48, 60, 4, 48, 12, 480, 72, 48, 72, 20]
})

df_cervezas.to_sql("cervezas", con=engine, index=False, if_exists="replace")
df_bares.to_sql("bares", con=engine, index=False, if_exists="replace")
df_empleados.to_sql("empleados", con=engine, index=False, if_exists="replace")
df_reparto.to_sql("reparto", con=engine, index=False, if_exists="replace")




11

In [12]:
# Obtener  el  nombre    de  los  empleados   que  hayan  repartido  al  bar  Stop  durante la semana del 17 al 23 de octubre de 2005.
query = """SELECT 
    emp.Nombre AS NombreEmpleado, 
    rep.Fecha, 
    b.Nombre AS NombreBar
FROM empleados AS emp
JOIN reparto AS rep ON emp.CodE = rep.CodE
JOIN bares AS b ON b.CodB = rep.CodB
WHERE b.Nombre = 'Stop'
  AND rep.Fecha > '10/17/05'
  AND rep.Fecha < '10/23/05';
"""

pd.read_sql(query, con=engine)

,NombreEmpleado,Fecha,NombreBar
0,Prudencio Caminero,10/21/05,Stop
1,Prudencio Caminero,10/21/05,Stop


In [13]:
#Obtener  el   Cif  y  nombre  de  los  bares  a  los  que  se  ha  repartido  cerveza  de  tipo  Botella  y  capacidad inferior a 1 litro, ordenados por localidad

query = """SELECT DISTINCT b.Cif, b.Nombre 
FROM bares AS b 
JOIN reparto AS rep ON b.CodB = rep.CodB 
JOIN cervezas as cerve ON rep.CodC = cerve.CodC 
WHERE cerve.Envase = 'Botella' 
AND cerve.Capacidad < 1 
ORDER BY b.Localidad
"""

pd.read_sql(query, con=engine)

,Cif,Nombre
0,11111111X,Stop
1,22222222Y,Las Vegas


In [14]:
# Obtener  los  repartos  (nombre  del  bar,  envase  y  capacidad  de  la  bebida, fecha y cantidad) realizados por Prudencio Caminero

query = """SELECT b.Nombre,c.Envase, c.Capacidad, rep.Fecha, rep.Cantidad
FROM reparto AS rep
JOIN bares AS b ON b.CodB = rep.CodB
JOIN cervezas AS c ON rep.CodC = c.CodC 
JOIN empleados AS emp ON emp.CodE = rep.CodE
WHERE emp.Nombre = 'Prudencio Caminero'
"""

pd.read_sql(query, con=engine)

,Nombre,Envase,Capacidad,Fecha,Cantidad
0,Stop,Botella,0.20,10/21/05,240
1,Stop,Botella,0.33,10/21/05,48
2,Las Vegas,Botella,0.20,10/22/05,4
3,Las Vegas,Botella,0.33,10/22/05,48
4,Las Vegas,Lata,0.33,10/22/05,12
5,Las Vegas,Lata,0.33,10/22/05,60


In [15]:
# Obtener  los  bares  a  los  que  se  les  ha  repartido  envases  de  tipo  botella  y  capacidad  0.2  ó 0.33

query = """
SELECT DISTINCT *
FROM bares AS b
JOIN reparto AS rep ON rep.CodB = b.CodB
JOIN cervezas AS c ON rep.CodC = c.CodC 
WHERE c.Capacidad > 0.2 OR c.Capacidad < 0.33
 


"""

pd.read_sql(query, con=engine)

,CodB,Nombre,Cif,Localidad,CodE,CodB,CodC,Fecha,Cantidad,CodC,Envase,Capacidad,Stock
0,001,Stop,11111111X,Villa Botijo,1,001,01,10/21/05,240,01,Botella,0.20,3600
1,002,Las Vegas,22222222Y,Villa Botijo,1,002,01,10/22/05,4,01,Botella,0.20,3600
2,001,Stop,11111111X,Villa Botijo,1,001,02,10/21/05,48,02,Botella,0.33,1200
3,002,Las Vegas,22222222Y,Villa Botijo,1,002,02,10/22/05,48,02,Botella,0.33,1200
4,002,Las Vegas,22222222Y,Villa Botijo,1,002,03,10/22/05,12,03,Lata,0.33,2400
5,002,Las Vegas,22222222Y,Villa Botijo,1,002,03,10/22/05,60,03,Lata,0.33,2400
6,003,Club Social,33333333Z,Las Ranas,2,003,03,10/24/05,48,03,Lata,0.33,2400
7,003,Club Social,33333333Z,Las Ranas,3,003,03,10/25/05,20,03,Lata,0.33,2400
8,002,Las Vegas,22222222Y,Villa Botijo,2,002,04,10/23/05,480,04,Botella,1.00,288
9,003,Club Social,33333333Z,Las Ranas,2,003,04,10/24/05,72,04,Botella,1.00,288


In [20]:
# Nombre de los empleados que han repartido a los bares "Stop" y "Las Vegas" cervezas con  envase botella.

query = """
SELECT DISTINCT emp.Nombre
FROM empleados AS emp 
JOIN reparto AS rep ON emp.CodE = rep.CodE
JOIN bares AS b ON b.CodB = rep.CodB
JOIN cervezas AS c ON rep.CodC = c.CodC
WHERE b.Nombre = 'Stop' OR  b.Nombre = 'Las Vegas' AND c.Envase = 'botella'
 
"""

pd.read_sql(query, con=engine)

,Nombre
0,Prudencio Caminero


In [24]:
# Obtener  el  nombre  y  número  de  viajes  que  ha  realizado  cada  empleado  fuera  de  Villa  Botijo.
query = """
SELECT DISTINCT emp.Nombre, COUNT(b.Localidad) AS viajes
FROM empleados AS emp 
JOIN reparto AS rep ON emp.CodE = rep.CodE
JOIN bares AS b ON b.CodB = rep.CodB
WHERE b.Localidad = 'Villa Botijo'
 
"""

pd.read_sql(query, con=engine)


,Nombre,viajes
0,Prudencio Caminero,8


In [28]:
# Obtener el nombre y localidad del bar que más litros de cerveza ha comprado.

query = """
SELECT DISTINCT b.Nombre, b.Localidad, MAX(c.Capacidad) AS capacidad
FROM bares AS b 
JOIN reparto AS rep ON b.CodB = rep.CodB
JOIN cervezas AS c ON rep.CodC = c.CodC

"""

pd.read_sql(query, con=engine)

,Nombre,Localidad,capacidad
0,Las Vegas,Villa Botijo,60.0


In [29]:
# Obtener  los  bares  que  han  adquirido  todos  los  tipos  de  cerveza  con  envase  de  botella   y capacidad menor que 1 litro.

query = """
SELECT b.Nombre
FROM bares AS b
WHERE NOT EXISTS (
    SELECT *
    FROM cervezas AS c
    WHERE c.Envase = 'Botella' AND c.Capacidad < 1
    AND NOT EXISTS (
        SELECT *
        FROM reparto AS r
        WHERE r.CodB = b.CodB AND r.CodC = c.CodC
    )
);

"""

pd.read_sql(query, con=engine)

,Nombre
0,Stop
1,Las Vegas


In [30]:
#Subir un 5% el sueldo del empleado que más días haya trabajado.

query = """
UPDATE empleados
SET Sueldo = Sueldo * 1.05
WHERE CodE = (
    SELECT rep.CodE
    FROM reparto AS rep
    GROUP BY rep.CodE
    ORDER BY COUNT(DISTINCT rep.Fecha) DESC
   
);

"""

pd.read_sql(query, con=engine)

ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [32]:
#  Insertar  un  nuevo  reparto  del  empleado  “Vicente Merario” al bar  “Stop” de 48 cervezas de tipo lata el día 10/26/05.

query = """
INSERT INTO reparto (CodE, CodB, CodC, Fecha, Cantidad)
VALUES (
    (SELECT CodE FROM empleados WHERE Nombre = 'Vicente Merario'),
    (SELECT CodB FROM bares WHERE Nombre = 'Stop'),
    (SELECT CodC FROM cervezas WHERE Envase = 'Lata'),
    '2005-10-26',
    48
);

"""

pd.read_sql(query, con=engine)

ResourceClosedError: This result object does not return rows. It has been closed automatically.